In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import geopandas as gpd
from shapely import wkt
from shapely.geometry import Point

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/시카고 결합x/Chicago_Crimes_2001_to_2004.csv', on_bad_lines='skip')

<ipython-input-5-17dbd816fe54>:1: DtypeWarning: Columns (17,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/시카고 결합x/Chicago_Crimes_2001_to_2004.csv', on_bad_lines='skip')


In [ ]:
df = df.dropna(subset=['X Coordinate','Y Coordinate','Latitude', 'Longitude', 'Location'])

In [ ]:
df.isna().sum()

,0
Unnamed: 0,0
ID,0
Case Number,0
Date,0
Block,0
IUCR,0
Primary Type,0
Description,0
Location Description,13
Arrest,0


In [ ]:
# 3. 범죄 데이터의 위도, 경도 정보를 사용해 GeoDataFrame 생성
df['geometry'] = df.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis=1)
crime_gdf = gpd.GeoDataFrame(df, geometry='geometry', crs="EPSG:4326")  # WGS84 좌표계 설정

In [ ]:
# 4. Community Area CSV 파일 불러오기 (the_geom을 WKT 형식으로 변환)
community_areas = pd.read_csv('/content/drive/MyDrive/CommAreas_20250325.csv')  # Community Area 데이터 (CSV)
community_areas['geometry'] = community_areas['the_geom'].apply(wkt.loads)  # the_geom을 WKT로 변환
community_areas_gdf = gpd.GeoDataFrame(community_areas, geometry='geometry', crs="EPSG:4326")

In [ ]:
# 5. Ward CSV 파일 불러오기 (the_geom을 WKT 형식으로 변환)
wards = pd.read_csv('/content/drive/MyDrive/Wards_2003_2015.csv')  # Ward 데이터 (CSV)
wards['geometry'] = wards['the_geom'].apply(wkt.loads)  # the_geom을 WKT로 변환
wards_gdf = gpd.GeoDataFrame(wards, geometry='geometry', crs="EPSG:4326")

In [ ]:
# 6. 범죄 데이터의 좌표가 어느 Ward에 속하는지 Spatial Join
crime_with_ward = gpd.sjoin(crime_gdf, wards_gdf[['WARD', 'geometry']], how='left', predicate='within')  # Ward 매핑

In [ ]:
wards_gdf.head()

,the_geom,DATA_ADMIN,PERIMETER,WARD,ALDERMAN,CLASS,WARD_PHONE,HALL_PHONE,HALL_OFFIC,ADDRESS,EDIT_DATE1,SHAPE_AREA,SHAPE_LEN,geometry
0,MULTIPOLYGON (((-87.61720985189288 41.84565509...,9.588452e+07,71516.462726,4,WILLIAM BURNS,3.0,773-536-8103,312-744-2690,"121 N LASALLE ST, RM 300 OFFICE 10, 60602",4659 S COTTAGE GROVE STE 203,20030527,9.693978e+07,73428.701824,"MULTIPOLYGON (((-87.61721 41.84566, -87.61692 ..."
1,MULTIPOLYGON (((-87.69441592800892 41.95563904...,6.278870e+07,46189.174036,33,RICHARD F. MELL,3.0,773-478-8040,312-744-6825,"121 N LASALLE ST, RM 208, 60602",3649 N KEDZIE AV,20020301,6.278870e+07,46189.173373,"MULTIPOLYGON (((-87.69442 41.95564, -87.69443 ..."
2,MULTIPOLYGON (((-87.6642015105961 42.021260262...,4.685230e+07,44816.944176,49,JOSEPH A. MOORE,5.0,773-338-5796,312-744-3067,"121 N LASALLE ST, RM 300 OFFICE 24, 60602",7356 N GREENVIEW AV,20030527,4.682849e+07,45091.156684,"MULTIPOLYGON (((-87.6642 42.02126, -87.66419 4..."
3,MULTIPOLYGON (((-87.77086308398384 41.92416619...,8.526330e+07,60411.795941,37,EMMA MITTS,2.0,773-745-2894,312-744-8019,"121 N. LASALLE ST, RM 300, 60602",5344 W NORTH AVE,20030527,8.526330e+07,60411.795499,"MULTIPOLYGON (((-87.77086 41.92417, -87.77085 ..."
4,MULTIPOLYGON (((-87.82815431483685 41.98357156...,5.490751e+06,16209.674429,OUT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.475219e+06,16252.000891,"MULTIPOLYGON (((-87.82815 41.98357, -87.82815 ..."


In [ ]:
# 7. 범죄 데이터의 좌표가 어느 Community Area에 속하는지 Spatial Join
crime_with_community = gpd.sjoin(crime_with_ward, community_areas_gdf[['COMMUNITY', 'geometry']], how='left', predicate='within', lsuffix='_ward', rsuffix='_community')  # Community Area 매핑

In [ ]:
# 8. 'Ward'와 'Community Area' 결측치 채우기
crime_with_community['Ward'] = crime_with_community['Ward'].fillna(crime_with_community['WARD'])
crime_with_community['Community Area'] = crime_with_community['Community Area'].fillna(crime_with_community['COMMUNITY'])

In [ ]:
# 9. 불필요한 열 삭제 (매핑된 'WARD', 'COMMUNITY' 열 제거)
crime_with_community = crime_with_community.drop(columns=['WARD', 'COMMUNITY','index_right','index__community'])

In [ ]:
crime_with_community = crime_with_community.drop(columns=['geometry'])

In [ ]:
# 10. 결측치 확인
print(crime_with_community[['Ward', 'Community Area']].isna().sum())  # 결측치 확인

Ward              493
Community Area    462
dtype: int64


In [ ]:
crime_with_community.isna().sum()

,0
Unnamed: 0,0
ID,0
Case Number,0
Date,0
Block,0
IUCR,0
Primary Type,0
Description,0
Location Description,13
Arrest,0


In [ ]:
# 11. 결과를 CSV로 저장
crime_with_community.to_csv('/content/drive/MyDrive/chicago_crime_data_with_ward_community1.csv', index=False)


In [ ]:
crime_with_community[['ID', 'Ward', 'Community Area']]

,ID,Ward,Community Area
1,4676906,11.0,61.0
4,4677901,34.0,49.0
6,4791194,9.0,50.0
7,4679521,21.0,73.0
9,4680124,24.0,29.0
...,...,...,...
1923510,4781176,37.0,19.0
1923511,4671197,38.0,15.0
1923512,4671380,18.0,71.0
1923513,4782588,10.0,46.0
